# DSCI 417 - Homework 01

**Malcolm M. Nichols**

In [0]:
import math as m
from pyspark.sql import SparkSession
from pyspark.mllib.random import RandomRDDs

In [0]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Problem 1: Terminology

1. Scala
2. Spark Session
3. Spark Context
4. Resilient Distributed Datasets
5. Partitions
6. Transformations
7. Action
8. Transformation
9. Action
10. Transformation
11. Action
12. List
13. Master Node
14. Worker Nodes
15. Driver
16. Executor

## Problem 2: Working with a Numerical RDD

In [0]:
random_rdd = RandomRDDs.uniformRDD(sc, size = 1200000, seed = 1)
print('Sum:     ', round(random_rdd.sum(),2))
print('Mean:    ', round(random_rdd.mean(),2))
print('Std Dev: ', round(random_rdd.stdev(),2))
print('Minimum: ', round(random_rdd.min(),2))
print('Maximum: ', round(random_rdd.max(),2))

Sum: 599075.07
Mean: 0.5
Std Dev: 0.29
Minimum: 0.0
Maximum: 1.0

In [0]:
partition_list = random_rdd.glom().map(len).collect()
print('Number of Partitions: ', random_rdd.getNumPartitions())
print('Size of Partitions:   ', partition_list)

Number of Partitions: 4
Size of Partitions: [300000, 300000, 300000, 300000]

## Problem 3: Transformations

In [0]:
scaled_rdd = random_rdd.map(lambda x : x * 10)

print('Sum:     ', round(scaled_rdd.sum(),2))
print('Mean:    ', round(scaled_rdd.mean(),2))
print('Std Dev: ', round(scaled_rdd.stdev(),2))
print('Minimum: ', round(scaled_rdd.min(),2))
print('Maximum: ', round(scaled_rdd.max(),2))

Sum: 5990750.66
Mean: 4.99
Std Dev: 2.89
Minimum: 0.0
Maximum: 10.0

In [0]:
log_rdd = scaled_rdd.map(lambda x : m.log(x))

print('Sum:     ', round(log_rdd.sum(),2))
print('Mean:    ', round(log_rdd.mean(),2))
print('Std Dev: ', round(log_rdd.stdev(),2))
print('Minimum: ', round(log_rdd.min(),2))
print('Maximum: ', round(log_rdd.max(),2))

Sum: 1559584.39
Mean: 1.3
Std Dev: 1.0
Minimum: -13.78
Maximum: 2.3

## Problem 4: Calculating SSE

In [0]:
pairs_raw = sc.textFile('/FileStore/tables/pairs_data.txt')
print(pairs_raw.count())

12743548

In [0]:
for ele in pairs_raw.take(5):
    print(ele)    

12.3 12.1
9.1 8.7
9.3 9.9
8.5 8.5
11.2 10.8

In [0]:
def process_line(row):
    items = row.split(' ')
    
    return (float(items[0]), float(items[1]))

pairs = pairs_raw.map(process_line)

for pair in pairs.take(4):
    print(pair)

(12.3, 12.1)
(9.1, 8.7)
(9.3, 9.9)
(8.5, 8.5)

In [0]:
SSE = pairs.map(lambda x : (x[0] - x[1]) ** 2).sum()
print(round(SSE,2))

4597380.19

## Problem 5: Calculating r-Squared

In [0]:
mean = pairs.map(lambda x : x[0]).mean()
print(round(mean,2))

10.0

In [0]:
SST = pairs.map(lambda x : (x[0] - mean) ** 2).sum()
print(round(SST,2))

24980514.86

In [0]:
r2 = 1 - (SSE / SST)
print(round(r2,2))

0.82

## Problem 6: NASA Server Logs

In [0]:
nasa = sc.textFile('/FileStore/tables/NASA_server_logs_Aug_1995.txt')
print(nasa.count())

1569888

In [0]:
for row in nasa.take(5):
    print(row)

in24.inetnebr.com [01/Aug/1995:00:00:01] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt" 200 1839
uplherc.upl.com [01/Aug/1995:00:00:07] "GET /" 304 0
uplherc.upl.com [01/Aug/1995:00:00:08] "GET /images/ksclogo-medium.gif" 304 0
uplherc.upl.com [01/Aug/1995:00:00:08] "GET /images/MOSAIC-logosmall.gif" 304 0
uplherc.upl.com [01/Aug/1995:00:00:08] "GET /images/USA-logosmall.gif" 304 0

In [0]:
get = nasa.map(lambda x : 'GET' in x).sum()
post = nasa.map(lambda x : 'POST' in x).sum()
head = nasa.map(lambda x : 'HEAD' in x).sum()

print('Number of GET requests:  ', get)
print('Number of POST requests: ', post)
print('Number of HEAD requests: ', head)

Number of GET requests: 1565812
Number of POST requests: 111
Number of HEAD requests: 3965